<a href="https://colab.research.google.com/github/ayushs0911/Practice/blob/main/IMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd 
import numpy as np 
import tensorflow as tf
import re 
import seaborn as sns 
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
import nltk 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:

!pip install -q kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
! kaggle datasets download -d atulanandjha/imdb-50k-movie-reviews-test-your-bert

mkdir: cannot create directory ‘/root/.kaggle’: File exists
 90% 23.0M/25.7M [00:00<00:00, 128MB/s] 
100% 25.7M/25.7M [00:00<00:00, 116MB/s]


In [9]:
!unzip /content/imdb-50k-movie-reviews-test-your-bert.zip

Archive:  /content/imdb-50k-movie-reviews-test-your-bert.zip
  inflating: test.csv                
  inflating: train.csv               


In [10]:
df = pd.read_csv("/content/train.csv")

In [11]:
df.head()

,text,sentiment
0,"Now, I won't deny that when I purchased this o...",neg
1,"The saddest thing about this ""tribute"" is that...",neg
2,Last night I decided to watch the prequel or s...,neg
3,I have to admit that i liked the first half of...,neg
4,I was not impressed about this film especially...,neg


In [12]:
def tokenize(df, input_col = "text", output_col = "tokenized_text"):
  df.loc[:, output_col] = df.loc[:, input_col].apply(lambda x : word_tokenize(x))
  return df

In [13]:
df = tokenize(df)

In [14]:
df.head()

,text,sentiment,tokenized_text
0,"Now, I won't deny that when I purchased this o...",neg,"[Now, ,, I, wo, n't, deny, that, when, I, purc..."
1,"The saddest thing about this ""tribute"" is that...",neg,"[The, saddest, thing, about, this, ``, tribute..."
2,Last night I decided to watch the prequel or s...,neg,"[Last, night, I, decided, to, watch, the, preq..."
3,I have to admit that i liked the first half of...,neg,"[I, have, to, admit, that, i, liked, the, firs..."
4,I was not impressed about this film especially...,neg,"[I, was, not, impressed, about, this, film, es..."


In [15]:
def remove_stopwords(words):
  filtered = [x for x in words if not x.lower() in stop_words]
  return filtered 

In [16]:
def punct(words):
  filtered = []
  for word in words:
    if re.findall("[()!><.,`?']", word):
      pass
    else:
      filtered.append(word)
  return filtered

In [17]:
def clean_text(df, input_col = 'tokenized_text', output_col = 'clean_text'):
  texts = df.loc[:, input_col]

  word = []
  for text in texts:
    t = remove_stopwords(text)
    t = punct(t)
    t = " ".join([str(i) for i in t])
    word.append(t)
  
  df.loc[:, output_col] = word
  return df

In [18]:
df = clean_text(df)

In [19]:
df.head()

,text,sentiment,tokenized_text,clean_text
0,"Now, I won't deny that when I purchased this o...",neg,"[Now, ,, I, wo, n't, deny, that, when, I, purc...",wo deny purchased eBay high expectations incre...
1,"The saddest thing about this ""tribute"" is that...",neg,"[The, saddest, thing, about, this, ``, tribute...",saddest thing tribute almost singers including...
2,Last night I decided to watch the prequel or s...,neg,"[Last, night, I, decided, to, watch, the, preq...",Last night decided watch prequel shall say cal...
3,I have to admit that i liked the first half of...,neg,"[I, have, to, admit, that, i, liked, the, firs...",admit liked first half Sleepers looked good ac...
4,I was not impressed about this film especially...,neg,"[I, was, not, impressed, about, this, film, es...",impressed film especially fact went cinema fam...


In [20]:
df.drop(columns = ['text', 'tokenized_text'], inplace= True)

In [21]:
df.head()

,sentiment,clean_text
0,neg,wo deny purchased eBay high expectations incre...
1,neg,saddest thing tribute almost singers including...
2,neg,Last night decided watch prequel shall say cal...
3,neg,admit liked first half Sleepers looked good ac...
4,neg,impressed film especially fact went cinema fam...


In [22]:
def sentiments(df, input_col = 'sentiment', output_col = 'target'):
  df.loc[:, output_col] = df.loc[:, input_col].apply(lambda x : 0 if x == "neg" else 1)
  return df

In [23]:
df = sentiments(df)

In [24]:
df.head()

,sentiment,clean_text,target
0,neg,wo deny purchased eBay high expectations incre...,0
1,neg,saddest thing tribute almost singers including...,0
2,neg,Last night decided watch prequel shall say cal...,0
3,neg,admit liked first half Sleepers looked good ac...,0
4,neg,impressed film especially fact went cinema fam...,0


In [25]:
df['target'].value_counts()

0    12500
1    12500
Name: target, dtype: int64

In [26]:
df.drop(columns = ['sentiment'], inplace = True)

In [27]:
from sklearn.model_selection import train_test_split

In [28]:
X_train, X_val, y_train, y_val = train_test_split(df['clean_text'].to_numpy(),
                                                  df['target'].to_numpy(),
                                                  test_size = 0.1,
                                                  random_state = 0)

In [29]:
len(X_train), len(X_val)

(22500, 2500)

In [30]:
X_train[:5]

array(['Return Jedi often remembered wrong rather right shame last chronological installment Star Wars saga shining example epic storytelling manages wrap story lines previous movies one grand finale br / br / Yes Ewoks - cute cuddly bears arguably served broaden Star Wars demographic - middle movie tends slow bit final hour arguably best piece entire saga Luke finally comes face face Darth Vader recognizable villain movie br / br / Return Jedi many things right people tend overlook : presented incredible conclusion Darth Vader storyline went slightly implausible Empire Strikes Back convincing exciting opening Jabba Palace masterful performance Ian McDiarmid Emperor Luke finally coming resolution Solo Leia romance extremely powerful final moments Endor br / br / Yes slight annoyances annoyances generation moviegoers time nitpick every single scene still magical moving piece cinema also serves great final chapter movie - fantastic',
       'Want know secret making slasher film set fitne

In [31]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization 

In [32]:
total_words = 0
for sent in X_train:
  total_words += len(sent.split())

In [33]:
total_words

2874246

In [34]:
output_sequence_length = total_words/len(X_train)
output_sequence_length

127.74426666666666

In [35]:
max_vocab_length = 20000
max_length = 128 # ~ 127.74426666

In [36]:
text_vectorizer = TextVectorization(max_tokens = max_vocab_length,
                                    output_mode = 'int', 
                                    output_sequence_length = max_length)

In [37]:
text_vectorizer.adapt(X_train)

In [38]:
import random 
random_sentance = random.choice(X_train)

In [39]:
print(random_sentance)
text_vectorizer([random_sentance])

Taiwanese director Ang Lee whose previous films include Sensibility Ice Storm turned American Civil War latest feature Based novel Daniel Woodrell follows exploits group Southern guerrillas known bushwhackers fight Northern equivalents jayhawkers backwater br / br / one might expect plenty visceral action focus tension war put young men fought - many fighting former neighbours even family Jake Roedel Tobey Maguire man rather boy seventeen war reaches Missouri son German immigrant instead following countrymen becoming Unionist joins lifelong friend Jack Bull Chiles Skeet Ulrich rides bushwhackers Despite lack acceptance ancestry unwillingness participate murder unarmed Union men remains loyal cause friend Daniel Holt Jeffrey Wright black slave freed another bushwhacker fighting br / br / Lee handles subject aplomb never rushing deep introspection plot demands favour action lends film sense reality war - long periods boredom waiting interposed occasional flashes intensely terrifying figh

<tf.Tensor: shape=(1, 128), dtype=int64, numpy=
array([[13157,    70, 13596,   629,   473,   791,    31,  1246,  7258,
         1897,  2906,   542,   201,  3226,   221,  2310,   668,   329,
          506,  2056,     1,   983,  5816,   458,  2663,     1,   515,
        16863,   406,  6034,     1,     1,     1,     2,     2,     5,
          125,   390,   796,  8892,   113,   986,   920,   221,   160,
           83,   226,  5083,    32,   830,   966, 10664,     9,   110,
         2804,     1, 17072,  9631,    47,   134,   304, 16336,   221,
         4036, 11275,   361,   990,  7174,   191,   885,     1,  1348,
            1,  4910,  8095,   319,   530,  4821,     1,     1, 16236,
         6187, 16863,   331,   433,  5478, 16139, 13645,  7144,   451,
        15436,  3408,   226,  1081,  4408,  1033,   319,  2056,  7813,
         3791,  6891,   211,  4299,  8811,    60,     1,   830,     2,
            2,   629,  5724,   722,  9528,    35, 12321,   793, 15798,
           38,  3670,  5029, 

In [40]:
from tensorflow.keras import layers 

In [41]:
embedding = layers.Embedding(input_dim = max_vocab_length, 
                             output_dim = 128, 
                             input_length = max_length)

In [42]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [43]:

from sklearn.metrics import accuracy_score, precision_recall_fscore_support
def calculate_results(y_true, y_pred):
  model_accuracy = accuracy_score(y_true, y_pred)*100
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average = 'weighted')
  model_results = {'accuracy' : model_accuracy,
                   'precision' : model_precision,
                   'recall' : model_recall,
                   'f1': model_f1}
  return model_results

In [44]:
inputs = layers.Input(shape = (1,), dtype = 'string')
x = text_vectorizer(inputs)
x = embedding(x)

x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation = 'sigmoid')(x)

model_1 = tf.keras.Model(inputs, outputs)


In [45]:
model_1.compile(loss = 'binary_crossentropy', 
                optimizer = tf.keras.optimizers.Adam(), 
                metrics = ['accuracy'])

In [46]:
history_1 = model_1.fit(X_train, y_train, 
                      epochs = 1, 
                      validation_data = (X_val, y_val))

704/704 [==============================] - 34s 47ms/step - loss: 0.5081 - accuracy: 0.8038 - val_loss: 0.3590 - val_accuracy: 0.8596


In [47]:
probs = model_1.predict(X_val)

79/79 [==============================] - 1s 7ms/step


In [48]:
preds = tf.squeeze(tf.round(probs))

In [49]:
preds

<tf.Tensor: shape=(2500,), dtype=float32, numpy=array([0., 0., 1., ..., 0., 1., 1.], dtype=float32)>

In [50]:
model_1_results = calculate_results(y_true = y_val, 
                                    y_pred = preds)

In [51]:
model_1_results


{'accuracy': 85.96000000000001,
 'precision': 0.8613304588097813,
 'recall': 0.8596,
 'f1': 0.8594681482005734}

# LSTM 

In [52]:
from tensorflow.keras import layers

In [53]:
embedding_2 = layers.Embedding(input_dim = max_vocab_length, 
                               output_dim = 128, 
                               input_length = max_length)

In [54]:
inputs = layers.Input(shape = (1,), dtype = 'string')
x = text_vectorizer(inputs)
x = embedding_2(x)

x = layers.LSTM(64)(x)
x = layers.Dense(64, activation = 'relu')(x)

outputs = layers.Dense(1, activation = 'sigmoid')(x)

model_2 = tf.keras.Model(inputs, outputs, name ="LSTM")



In [55]:
model_2.compile(loss = 'binary_crossentropy', 
                optimizer = tf.keras.optimizers.Adam(), 
                metrics = ['accuracy'])

In [56]:
history_2 = model_2.fit(X_train, y_train, 
                      epochs = 1, 
                      validation_data = (X_val, y_val))

704/704 [==============================] - 159s 215ms/step - loss: 0.6603 - accuracy: 0.5908 - val_loss: 0.5281 - val_accuracy: 0.7792


In [57]:
model_2_probs = model_2.predict(X_val)
model_2_preds = tf.squeeze(tf.round(model_2_probs))

79/79 [==============================] - 6s 50ms/step


In [58]:
model_2_results = calculate_results(y_true = y_val, 
                                    y_pred = model_2_preds)

In [59]:
model_2_results

{'accuracy': 77.92,
 'precision': 0.7796664385931193,
 'recall': 0.7792,
 'f1': 0.7791457272999609}

#GRU

In [60]:
from tensorflow.keras import layers

In [61]:
embedding_3 = layers.Embedding(input_dim = max_vocab_length, 
                               output_dim = 128, 
                               input_length = max_length)

In [62]:
inputs = layers.Input(shape = (1,), dtype = 'string')
x = text_vectorizer(inputs)
x = embedding_3(x)

x = layers.GRU(64)(x)
x = layers.Dense(32, activation = 'relu')(x)

outputs = layers.Dense(1, activation = 'sigmoid')(x)

model_3 = tf.keras.Model(inputs, outputs, name = "model_3_GRU")


In [63]:
model_3.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

In [64]:
history_3 = model_3.fit(X_train, y_train, 
                        epochs =1,
                        validation_data = (X_val, y_val))

704/704 [==============================] - 116s 160ms/step - loss: 0.6710 - accuracy: 0.5504 - val_loss: 0.4237 - val_accuracy: 0.8220


# Bidirectional RNN model 

In [67]:
embedding_4 = layers.Embedding(input_dim = max_vocab_length, 
                               output_dim = 128, 
                               embeddings_initializer = 'uniform',
                               input_length = max_length)

In [68]:
inputs = layers.Input(shape = (1,), dtype = 'string')
x = text_vectorizer(inputs)
x = embedding_4(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
outputs = layers.Dense(1, activation = 'sigmoid')(x)

model_4 = tf.keras.Model(inputs, outputs,)

In [69]:
model_4.compile(loss='binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

#Conv1D 

In [71]:
from tensorflow.keras import layers 
embeddoing_5 = layers.Embedding(input_dim = max_vocab_length, 
                                 output_dim = 128, 
                                 input_length = max_length)

In [ ]:
inputs = layers.Input(shape = (1,), dtype = 'string')
x = text_vectorizer(inputs)
x = embedding_5(x)

x = layers.Conv1D(filters = 32, kernel_size = 5, activation = 'relu')(x)
x = layers.GlobalMaxPool1D()(x)

outputs = layers.Dense(1, activation = 'relu')(x)
model_5 = tf.keras.Model(inputs, outputs)

model_5.compile(loss = 'binary_crossentropy()')

In [72]:
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4") # load Universal Sentence Encoder
embed_samples = embed(["When you call the universal sentence encoder on a sentence, it turns it into numbers."])

print(embed_samples[0][:50])


tf.Tensor(
[ 0.03596691 -0.08579469 -0.01152743  0.00525982 -0.01852172 -0.05042012
 -0.03616941  0.00534677  0.04805917  0.04690744 -0.0372333  -0.01149546
  0.04352415  0.07050991  0.0709376  -0.08180431  0.00871717 -0.04654121
 -0.02245776  0.04686872  0.00202256  0.03099072  0.02043563  0.06392168
 -0.07641086  0.08421179 -0.04576042 -0.00106165 -0.02059416  0.01241106
  0.05727539  0.0381562  -0.02742118 -0.00354347 -0.09832586 -0.01244854
  0.0386563   0.05031953 -0.02362506  0.00321848  0.03225209  0.00738095
  0.04473104 -0.00412236  0.01151601  0.02837724  0.00060139 -0.05903354
 -0.00494347 -0.00688527], shape=(50,), dtype=float32)


In [73]:

sentance_encoded_layer = hub.KerasLayer('https://tfhub.dev/google/universal-sentence-encoder/4',
                                        input_shape = [],
                                        dtype = tf.string,
                                        trainable = False, 
                                        name = 'USE')

In [74]:
model_6 = tf.keras.Sequential([
    sentance_encoded_layer,
    layers.Dense(64, activation = 'relu'),
    layers.Dense(1, activation = 'sigmoid')
], name = 'model_6')

model_6.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])